In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
train1 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/train1_prep.csv')
train2 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/train2_prep.csv')
train3 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/train3_prep.csv')
train4 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/train4_prep.csv')

test1 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/test1_prep.csv')
test2 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/test2_prep.csv')
test3 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/test3_prep.csv')
test4 = pd.read_csv('/content/drive/MyDrive/LG Aimers/data/test4_prep.csv')

In [ ]:
y_train = train1.copy()['target']

# 타겟변수 제거

In [ ]:
train1.drop('target',axis=1,inplace=True)
train2.drop('target',axis=1,inplace=True)
train3.drop('target',axis=1,inplace=True)
train4.drop('target',axis=1,inplace=True)

# 분할 작업 검증

In [ ]:
print(len(train1))
print(len(train2))
print(len(train3))
print(len(train4))

40506
40506
40506
40506


In [ ]:
print(len(test1))
print(len(test2))
print(len(test3))
print(len(test4))

17361
17361
17361
17361


In [ ]:
print(len(train1.columns)==len(test1.columns))
print(len(train2.columns)==len(test2.columns))
print(len(train3.columns)==len(test3.columns))
print(len(train4.columns)==len(test4.columns))

True
True
True
True


# 데이터 병합

In [ ]:
X_train = pd.concat([train1,train2,train3,train4],axis=1)
test = pd.concat([test1,test2,test3,test4],axis=1)

# 중복 열 확인

In [ ]:
duplicate_columns = X_train.T.duplicated()

In [ ]:
X_train.columns[duplicate_columns]

Index(['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
       'CURE END POSITION X Collect Result_Dam_1000.0',
       'CURE END POSITION Z Collect Result_Dam_12.5',
       'CURE END POSITION Θ Collect Result_Dam_90',
       'CURE START POSITION Θ Collect Result_Dam_90',
       'HEAD Standby Position Z Collect Result_Dam_66',
       'Head Clean Position Y Collect Result_Dam_127.5',
       'Head Purge Position Y Collect Result_Dam_257',
       'Head Zero Position X Collect Result_Dam_505.0',
       'Stage1 Circle3 Distance Speed Collect Result_Dam',
       'Stage1 Circle4 Distance Speed Collect Result_Dam',
       'Stage1 Line3 Distance Speed Collect Result_Dam',
       'Stage2 Circle3 Distance Speed Collect Result_Dam',
       'Stage2 Circle4 Distance Speed Collect Result_Dam',
       'Stage2 Line1 Distance Speed Collect Result_Dam',
       'Stage3 Circle3 Distance Speed Collect Result_Dam',
       'Stage3 Circle4 Distance Speed Collect Result_Dam',
       'Stage3 Line3 Dist

In [ ]:
df = X_train.copy()
for i in range(len(df.columns)):
    for j in range(i + 1, len(df.columns)):
        if df.iloc[:, i].equals(df.iloc[:, j]):
            print(f"{df.columns[i]} ///// {df.columns[j]}")

Workorder_Dam ///// Workorder_AutoClave
Workorder_Dam ///// Workorder_Fill1
HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam ///// HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam
Equipment_Dam_Dam dispenser #2 ///// CURE END POSITION X Collect Result_Dam_1000.0
Equipment_Dam_Dam dispenser #2 ///// CURE END POSITION Z Collect Result_Dam_12.5
Equipment_Dam_Dam dispenser #2 ///// CURE END POSITION Θ Collect Result_Dam_90
Equipment_Dam_Dam dispenser #2 ///// CURE START POSITION Θ Collect Result_Dam_90
Model.Suffix_Dam_AJX75334502 ///// Model.Suffix_AutoClave_AJX75334502
Model.Suffix_Dam_AJX75334502 ///// Model.Suffix_Fill1_AJX75334502
Model.Suffix_Dam_AJX75334502 ///// Model.Suffix_Fill2_AJX75334502
Model.Suffix_Dam_AJX75334503 ///// Model.Suffix_AutoClave_AJX75334503
Model.Suffix_Dam_AJX75334503 ///// Model.Suffix_Fill1_AJX75334503
Model.Suffix_Dam_AJX75334503 ///// Model.Suffix_Fill2_AJX75334503
Model.Suffix_Dam_AJX75334505 ///// Model.Suffix_AutoClave_AJX75334505
Model.

# 중복 열 제거

In [ ]:
X_train = X_train.loc[:, ~duplicate_columns]

In [ ]:
test = test.loc[:, ~duplicate_columns]

# 2차 검증

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 120 entries, Workorder_Dam to WorkMode Collect Result_Fill2_17
dtypes: float64(77), int64(42), object(1)
memory usage: 37.1+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 120 entries, Workorder_Dam to WorkMode Collect Result_Fill2_17
dtypes: float64(78), int64(41), object(1)
memory usage: 15.9+ MB


In [ ]:
(X_train.columns != test.columns).sum()

0

## dtype이 다른 열 존재(Workorder_Fill2_encoded)
이 열은 빈도 인코딩한 것으로, 사실 유효성이 없어보임.

In [ ]:
int64_columns_train = X_train.select_dtypes(include=['int64'])
print(int64_columns_train.columns)

Index(['CURE SPEED Collect Result_Dam',
       'DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'Production Qty Collect Result_Dam', 'Receip No Collect Result_Dam',
       'Stage1 Circle1 Distance Speed Collect Result_Dam',
       'Stage1 Circle2 Distance Speed Collect Result_Dam',
       'Stage1 Line1 Distance Speed Collect Result_Dam',
       'Stage1 Line2 Distance Speed Collect Result_Dam',
       'Stage1 Line4 Distance Speed Collect Result_Dam',
       'Stage2 Circle1 Distance Speed Collect Result_Dam',
       'Stage2 Circle2 Distance Speed Collect Result_Dam',
       'Stage2 Line2 Distance Speed Collect Result_Dam',
       'Stage2 Line3 Distance Speed Collect Result_Dam',
       'Stage2 Line4 Distance Speed Collect Result_Dam',
       'Stage3 Circle1 Distance Speed Collect Result_Dam',
       'Stage3 Circle2 Distance Speed Collect Result_Dam',
       'Stage3 Line1 Distance Speed Collect Result_Dam',
       'Stage3 Line2 Distance Speed Collect Result_Dam',
       'Stage3 Line

In [ ]:
int64_columns_test = test.select_dtypes(include=['int64'])
print(int64_columns_test.columns)

Index(['CURE SPEED Collect Result_Dam',
       'DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'Production Qty Collect Result_Dam', 'Receip No Collect Result_Dam',
       'Stage1 Circle1 Distance Speed Collect Result_Dam',
       'Stage1 Circle2 Distance Speed Collect Result_Dam',
       'Stage1 Line1 Distance Speed Collect Result_Dam',
       'Stage1 Line2 Distance Speed Collect Result_Dam',
       'Stage1 Line4 Distance Speed Collect Result_Dam',
       'Stage2 Circle1 Distance Speed Collect Result_Dam',
       'Stage2 Circle2 Distance Speed Collect Result_Dam',
       'Stage2 Line2 Distance Speed Collect Result_Dam',
       'Stage2 Line3 Distance Speed Collect Result_Dam',
       'Stage2 Line4 Distance Speed Collect Result_Dam',
       'Stage3 Circle1 Distance Speed Collect Result_Dam',
       'Stage3 Circle2 Distance Speed Collect Result_Dam',
       'Stage3 Line1 Distance Speed Collect Result_Dam',
       'Stage3 Line2 Distance Speed Collect Result_Dam',
       'Stage3 Line

In [ ]:
test['Workorder_Fill2_encoded'].unique() # 결측 존재(기존 train 인코딩 시에 없던 데이터)

array([ 87.,  46., 164., 175.,  36.,  61.,  62., 159.,  44.,  65., 170.,
       105.,  15., 124., 112.,  63.,  72.,  70.,  42.,  71., 149.,  30.,
       160.,  13., 145.,  40.,  54.,  73.,  69.,  33., 103.,  68.,  55.,
       128., 144.,  51.,  25., 171.,  45.,  84., 165., 106.,  95., 147.,
       178., 117.,  43., 167.,  90.,  99., 101.,  60.,  66., 142.,  31.,
       114.,  67.,  76.,  80., 107.,  32., 100., 104.,  83.,  56.,  64.,
       272.,  38., 155.,  28.,  85.,  48.,  27., 109.,  57., 157.,  78.,
        18.,  22.,  52.,  92.,  50.,  77., 133., 121., 176., 125.,  34.,
        53.,  59.,  74.,  11.,  35., 116., 108.,  86., 156.,   4., 189.,
        19.,  88.,  26.,  39., 227.,  14.,  37.,  98., 129.,  89.,  29.,
       131.,  21., 161., 217., 135., 182.,  75.,  91., 168., 169., 181.,
       153.,  16.,   7.,  41., 139.,  10.,  20.,  17.,  82.,   2., 163.,
        12., 183., 143.,  81.,  24.,   6.,  49.,  97.,   5.,  23.,  58.,
        47.,  94., 102., 113.,   8.,   9.,   3.,  n

In [ ]:
X_train.drop('Workorder_Fill2_encoded',axis=1,inplace=True)
test.drop('Workorder_Fill2_encoded',axis=1,inplace=True)

In [ ]:
object_columns_train = X_train.select_dtypes(include=['object'])
object_columns_test = test.select_dtypes(include=['object'])

In [ ]:
print(object_columns_train.columns)
print(object_columns_test.columns)

Index(['Workorder_Dam'], dtype='object')
Index(['Workorder_Dam'], dtype='object')


In [ ]:
X_train.drop('Workorder_Dam',axis=1,inplace=True)
test.drop('Workorder_Dam',axis=1,inplace=True)

# 중복 행 확인

In [ ]:
X_train[X_train.duplicated()]

,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,...,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,Equipment_Fill2_Fill2 dispenser #2,WorkMode Collect Result_Fill2_1,WorkMode Collect Result_Fill2_3,WorkMode Collect Result_Fill2_6,WorkMode Collect Result_Fill2_9,WorkMode Collect Result_Fill2_17


# 마지막 검증

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 118 entries, CURE SPEED Collect Result_Dam to WorkMode Collect Result_Fill2_17
dtypes: float64(77), int64(41)
memory usage: 36.5 MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 118 entries, CURE SPEED Collect Result_Dam to WorkMode Collect Result_Fill2_17
dtypes: float64(77), int64(41)
memory usage: 15.6 MB


In [ ]:
import pandas as pd

# 데이터프레임의 열 이름 집합 가져오기
test_columns = set(test.columns)
X_train_columns = set(X_train.columns)

# test에만 있는 열 이름
test_unique_columns = test_columns - X_train_columns

# X_train에만 있는 열 이름
X_train_unique_columns = X_train_columns - test_columns

# 결과 출력
print("test에만 있는 열들:")
print(test_unique_columns)
print("-" * 40)

print("X_train에만 있는 열들:")
print(X_train_unique_columns)
print("-" * 40)

test에만 있는 열들:
set()
----------------------------------------
X_train에만 있는 열들:
set()
----------------------------------------


# 저장

In [ ]:
train = pd.concat([X_train,y_train],axis=1)

In [ ]:
train.to_csv('/content/drive/MyDrive/LG Aimers/data/train_prep.csv',index=False)
test.to_csv('/content/drive/MyDrive/LG Aimers/data/test_prep.csv',index=False)